In [1]:
url = 'https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer/8?f=pjson'

In [2]:
import requests
import geopandas as gpd
import json
import itertools
from pprint import pprint
from shapely.geometry import shape

# Define the service URL for the feature layer
service_url = "https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer/8/query"
xmin, ymin = 150.95, -34.1183  # Lower-left corner

for x, y in itertools.product(range(0, 10, 1), range(0, 10, 1)): 
    # Define the parameters for the query
    params = {
        'where': '1=1',  # Query condition, here it selects all features
        'geometry': json.dumps({
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmin + 0.1 * x,
            'ymax': ymin + 0.1 * y,
            'spatialReference': {'wkid': 4326}
        }),
        'geometryType': 'esriGeometryEnvelope',
        'inSR': '4326',  # Spatial reference of the input geometry (WGS84)
        'spatialRel': 'esriSpatialRelIntersects',  # Spatial relationship
        'outFields': '*',  # Output fields
        'outSR': '4326',  # Spatial reference of the output
        'f': 'json'  # Format of the output
    }
    
    # Make the GET request to the API
    response = requests.get(service_url, params=params)
    
    # Check the status of the request
    if response.status_code == 200:
        data = response.json()
        geometries, attributes = [], []
        for f in data.get('features', []):
            geometry = f['geometry']
            if 'rings' in geometry:
                geom = shape({"type": "Polygon", "coordinates": geometry['rings']})
            elif 'paths' in geometry:
                geom = shape({"type": "LineString", "coordinates": geometry['paths']})
            else:
                geom = shape(geometry)
            geometries.append(geom)
            attributes.append(f['attributes'])
gdf = gpd.GeoDataFrame(attributes, geometry=geometries, crs="EPSG:4326")
gdf.plot()
display(gdf.head())
display(gdf.info())


KeyboardInterrupt



In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import json

# Define the service URL for the feature layer
service_url = "https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer/12/query"

# Define the bounding box coordinates for Sydney
xmin, ymin = 150.9516, -34.1183  # Lower-left corner
xmax, ymax = 151.2759, -33.5781  # Upper-right corner

# Define the parameters for the query
params = {
    'where': '1=1',  # Query condition, here it selects all features
    'geometry': json.dumps({
        'xmin': xmin,
        'ymin': ymin,
        'xmax': xmax,
        'ymax': ymax,
        'spatialReference': {'wkid': 4326}
    }),
    'geometryType': 'esriGeometryEnvelope',
    'inSR': '4326',  # Spatial reference of the input geometry (WGS84)
    'spatialRel': 'esriSpatialRelIntersects',  # Spatial relationship
    'outFields': '*',  # Output fields
    'outSR': '4326',  # Spatial reference of the output
    'f': 'json'  # Format of the output
}

# Make the GET request to the API
response = requests.get(service_url, params=params)

# Check the status of the request
if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Request failed with status code {response.status_code}")

# Extract features from the JSON response
features = data.get('features', [])

# Initialize lists to hold geometries and attributes
geometries = []
attributes = []

# Process each feature
for feature in features:
    geometry = feature.get('geometry')
    if geometry:
        if 'rings' in geometry:
            geom = shape({"type": "Polygon", "coordinates": geometry['rings']})
        elif 'paths' in geometry:
            geom = shape({"type": "LineString", "coordinates": geometry['paths']})
        else:
            geom = shape(geometry)
        geometries.append(geom)
        attributes.append(feature['attributes'])

# Create a GeoDataFrame
if geometries:
    gdf = gpd.GeoDataFrame(attributes, geometry=geometries, crs="EPSG:4326")
    gdf.plot()
    display(gdf.info())
    display(gdf[['address']].head())
else:
    print("No valid geometries found.")


In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import json

# Define the service URL for the feature layer
service_url = "https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Land_Parcel_Property_Theme/FeatureServer/12/query"

# Define the bounding box coordinates for Sydney
xmin, ymin = 150.9516, -34.1183  # Lower-left corner
xmax, ymax = 151.2759, -33.5781  # Upper-right corner

# Define the grid size (degrees)
grid_size = 0.01  # Adjust this value for finer or coarser grid

# Create lists to hold the results
all_geometries = []
all_attributes = []

# Iterate over the grid
x = xmin
while x < xmax:
    y = ymin
    while y < ymax:
        # Define the current grid cell
        cell_xmin = x
        cell_ymin = y
        cell_xmax = min(x + grid_size, xmax)
        cell_ymax = min(y + grid_size, ymax)

        # Define the parameters for the query
        params = {
            'where': '1=1',  # Query condition, here it selects all features
            'geometry': json.dumps({
                'xmin': cell_xmin,
                'ymin': cell_ymin,
                'xmax': cell_xmax,
                'ymax': cell_ymax,
                'spatialReference': {'wkid': 4326}
            }),
            'geometryType': 'esriGeometryEnvelope',
            'inSR': '4326',  # Spatial reference of the input geometry (WGS84)
            'spatialRel': 'esriSpatialRelIntersects',  # Spatial relationship
            'outFields': '*',  # Output fields
            'outSR': '4326',  # Spatial reference of the output
            'f': 'json'  # Format of the output
        }

        # Make the GET request to the API
        response = requests.get(service_url, params=params)

        # Check the status of the request
        if response.status_code == 200:
            data = response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
            y += grid_size
            continue

        # Extract features from the JSON response
        features = data.get('features', [])

        # Process each feature
        for feature in features:
            geometry = feature.get('geometry')
            if geometry:
                try:
                    # Verify and process the geometry
                    if 'rings' in geometry:
                        geom = shape({"type": "Polygon", "coordinates": geometry['rings']})
                    elif 'paths' in geometry:
                        geom = shape({"type": "LineString", "coordinates": geometry['paths']})
                    else:
                        geom = shape(geometry)
                    all_geometries.append(geom)
                    all_attributes.append(feature['attributes'])
                except (TypeError, ValueError) as e:
                    print(f"Skipping invalid geometry: {geometry} due to error: {e}")
            else:
                print(f"Feature without geometry: {feature}")

        # Move to the next grid cell in the y direction
        y += grid_size

    # Move to the next grid cell in the x direction
    x += grid_size

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(all_attributes, geometry=all_geometries, crs="EPSG:4326")
gdf.plot()

In [ ]:
gdf.to_crs("EPSG:7844").plot()

In [ ]:
gdf.info()

In [ ]:
from lib.gnaf_db import GnafDb

gnaf = GnafDb.create()
gnaf.get_container()